#Exploring Patterns of Population Structure and Environmental Associations to Aridity Across the Range of Loblolly Pine

##Introduction

In this set of analyses, we will be making use of data from the Eckert et al. 2010 paper to explore patterns of phenotypic and environmental associations among populations of loblolly pine.


###Abstract

Natural populations of forest trees exhibit striking phenotypic adaptations to diverse environmental
gradients, thereby making them appealing subjects for the study of genes underlying ecologically relevant phenotypes. Here, we use a genome-wide data set of single nucleotide polymorphisms genotyped across 3059 functional genes to study patterns of population structure and identify loci associated with aridity across the natural range of loblolly pine (Pinus taeda L.). Overall patterns of population structure, as inferred using principal components and Bayesian cluster analyses, were consistent with three genetic clusters likely resulting from expansions out of Pleistocene refugia located in Mexico and Florida. A novel application of association analysis, which removes the confounding effects of shared ancestry on correlations between genetic and environmental variation, identified five loci correlated with aridity. These loci were primarily involved with abiotic stress response to temperature and drought. A unique set of 24 loci was identified as FST outliers on the basis of the genetic clusters identified previously and after accounting for expansions out of Pleistocene refugia. These loci were involved with a diversity of physiological processes. Identification of nonoverlapping sets of loci highlights the fundamental differences implicit in the use of either method and suggests a pluralistic, yet complementary, approach to the identification of genes underlying ecologically relevant phenotypes.


##Overview of tasks

In general, what you will be doing is working your way from loading and saving data related to this study, to corrections for population structure, to looking for associations between genotypes and phenotypes, genotypes and the environment (`Bayenv2`), and genotypes+phenotypes+environment (`SQUAT`)

## This notebook

This notebook will take you through executing Principal Components Analysis on the data, checking for outliers (and fixing them) and determining how many PCA axes to use for downstream correction of population structure.

You will:

1. Interface between Python and R to execute some commands
1. Get some experience with plotting in matplotlib
1. Create and save PCA axes
1. Learn about applying commands to `DataFrames`, similar to `R`'s (`apply`, `sapply`, `mapply`, etc)

As with the previous notebook, execute the cell with the imports and continue

In [0]:
import os, sys
from IPython.display import Image
import pandas as pd
from __future__ import division
import numpy as np
import rpy2
from rpy2 import robjects as ro
import pandas.rpy.common as com
import matplotlib.pyplot as plt
import seaborn as sns
import operator
import scipy as sp
import traceback
from sklearn import preprocessing
from IPython.parallel import Client
from subprocess import Popen, PIPE
import shutil
from IPython.display import FileLink, FileLinks, Image
import psutil
import multiprocessing
from hdfstorehelper import HDFStoreHelper
import warnings
import pandas
import dill
warnings.simplefilter("ignore", pandas.io.pytables.PerformanceWarning)
%matplotlib inline

%load_ext rpy2.ipython
pd.set_option('display.width', 80)
pd.set_option('max.columns', 30)

%load_ext autoreload
%autoreload 2

sns.set_context("talk")

####Set up some functions again

In [0]:
def is_homozygous(gt):
    if len(set([x.strip() for x in gt.split("/")])) == 1:
        return True
    return False


def get_allele_counts(counts):
    a = {}
    het = 0
    for gt in counts.index:
        for allele in [x.strip() for x in gt.split("/")]:
            if not allele in a:
                a[allele] = 0
            a[allele] += counts[gt]
        if not is_homozygous(gt):
            het += counts[gt]
    return sorted(a.items(), key=lambda x: x[1], reverse=True), het


def get_correction(n):
    #for finite sample size
    return (2*n)/(2*n-1)


def get_allele_freqs(locus):
    locus = locus[locus != '?/?']
    locus = locus[locus != 'NA']
    c = locus.value_counts()
    c = c.sort(inplace=False, ascending=False)
    allele_counts = get_allele_counts(c)
    total_alleles = 2.0*sum(c)
    num_individuals = sum(c)
    A = ""
    a = ""
    P = 0
    Q = 0
    if len(allele_counts[0]) == 2:
        A = allele_counts[0][0][0]
        a = allele_counts[0][1][0]
        P = allele_counts[0][0][1]
        Q = allele_counts[0][1][1]
    else:
        A = allele_counts[0][0][0]
        P = P = allele_counts[0][0][1]
    PQ = allele_counts[-1]
    p = P/total_alleles
    q = Q/total_alleles
    assert p + q == 1.0
    He = 2 * p * q * get_correction(num_individuals)
    Ho = PQ*1.0/num_individuals
    Fis = 1 - (Ho/He)
    #print p, q, He, Ho, Fis
    ret = pd.Series({"p":p, 
                      "q":q,
                      "P":P,
                      "Q":Q,
                      "He":He,
                      "Ho":Ho, 
                      "Fis":Fis,
                    "PQ": PQ,
                    "total_alleles":total_alleles,
                    "num_indiv":num_individuals,
                    "A":A,
                    "a":a})
    return ret


def plot_hist(df, index):
    d = df.ix[index,:]
    plt.hist(d, bins=20)
    plt.title("%s %.2f $\pm$ %.3f [%.2f, %.2f]" % (index, 
                                                   np.mean(d), 
                                                   np.std(d),
                                                  np.min(d),
                                                  np.max(d)))
    

def convert_to_z12(locus):
    freq = af[locus.name]
    trans = {"%s/%s" % (freq["A"],freq["A"]): 0,
            "%s/%s" % (freq["a"],freq["a"]): 2,
            "%s/%s" % (freq["A"],freq["a"]): 1,
            "%s/%s" % (freq["a"],freq["A"]): 1,
            "?/?":-1}
    return locus.apply(lambda x: trans[x])


def center_and_standardize_value(val, u, var):
    if val == -1:
        return 0.0
    return (val-u)/np.sqrt(var)


def center_and_standardize(snp):
    maf = af.ix["q",snp.name]
    u = np.mean([x for x in snp if x != -1])
    var = np.sqrt(maf*(1-maf))
    return snp.apply(center_and_standardize_value, args=(u, var))

In [0]:
hdf = HDFStoreHelper("data.hd5")

####The cell below sets up the interface between Pythong and R.  You'll see.  It's slick.

In [0]:
r = ro.r
prcomp = r('prcomp')
summary = r('summary')

####The following cells will perform:

1. retrieval of the normalized genotypes for PCA
1. Sending that data to R, running `prcomp` and saving the results back in Python
1. Extracting individual component data from the `prcomp_res` object.
1. Plotting PC1 vs PC2

In [0]:
pca_std = hdf.get('pca_std')
prcomp_res = prcomp(pca_std, scale=False, center=False)

In [0]:
print(summary(prcomp_res))

In [0]:
x = com.convert_robj(prcomp_res.rx2("x"))
x.index = pca_std.index
x.ix[0:5,0:10]

In [0]:
plt.scatter(x.PC1, x.PC2)
plt.show()

####Notice anything about this plot?

* Does it look like you would expect a PCA plot to look like?
* In the paper, they do end up making some corrections for outliers.

####Just for fun, let's see how many axes might describe the population structure.

In [0]:
%%R
source("tw_calc.R")
test=read.table("twtable", header=F)

In [0]:
TWcalc = r('TWcalc')
tw = TWcalc(com.convert_to_r_matrix(pca_std), 25)
tw_p = com.convert_robj(tw.rx2(2))
tw_e = com.convert_robj(tw.rx2(1))
tw_num = 0
for i, p in enumerate(tw_p):
    print i, p
    if p > 0.05:
        tw_num = i
        break
print "Tracy-Widom test yields %d axes of pop structure" % tw_num

####Hmmm, that seems like a lot.  Let's fix it with some outlier exclusion.  

Can you describe what might be happening here?  If not, maybe it's in the paper...

In [0]:
y = pd.DataFrame(x)
for col in y.columns[0:10]:
    s_cutoff = np.std(y[col])*6
    u = np.mean(y[col])
    cutoff = sorted([u+s_cutoff, u-s_cutoff], reverse=True)
    outliers = y[col][(y[col] > cutoff[0]) | (y[col] < cutoff[1])]
    print col
    print outliers
    y = y.drop(outliers.index)
y.ix[0:5,0:10]

In [0]:
genotypes = hdf.get("genotypes")
gt_drop = genotypes.ix[y.index,:]
gt_drop.head()

In [0]:
af = hdf.get("af")
z12_drop = gt_drop.apply(convert_to_z12)
z12_drop.head()

In [0]:
pca_drop_std = z12_drop.apply(center_and_standardize)

In [0]:
hdf.put('pca_drop_std', pca_drop_std)

In [0]:
prcomp_res_drop = prcomp(pca_drop_std, scale=False, center=False)

In [0]:
x_drop = com.convert_robj(prcomp_res_drop.rx2("x"))
x_drop.index = pca_drop_std.index
x_drop.ix[0:5,0:5]

In [0]:
plt.scatter(x_drop.PC1, x_drop.PC2)
plt.show()

In [0]:
print summary(prcomp_res_drop)

In [0]:
tw = TWcalc(com.convert_to_r_matrix(pca_drop_std), 25)
tw_p = com.convert_robj(tw.rx2(2))
tw_e = com.convert_robj(tw.rx2(1))
tw_num = 0
for i, p in enumerate(tw_p):
    print i, p
    if p > 0.05:
        tw_num = i
        break
print "Tracy-Widom test yields %d axes of pop structure" % tw_num

####OK, that was a lot of work you just did.  To recap:

1. We found the outliers based on some criteria (Did you figure it out/look it up?)
1. We removed those outliers from the raw base/base data
1. We normalized the data again, performed PCA, and tested with T-W.  

How many axes of pop structure are left now?  Does that seem reasonable?

####The next section deals with hierfstat.  Sam already talked a little bit about this, but...

1. We have to transform our data again to their format
1. We're going to us the 012 data that does not contain outliers.
1. We're going to stash a copy of the input data for later use with `Bayenv`.  I do this because I care about you.
1. Because hierfstat knows about populations, we need to get that data in there, too.  Run the next few cells until that's done.  You'll know it's done when you write the `hierf.txt` file.

In [0]:
hierf_trans = {0:11, 1:12, 2:22, -1:'NA'}
def apply_hierf_trans(series):
    return [hierf_trans[x] if x in hierf_trans else x for x in series]

In [0]:
hierf_df = z12_drop.apply(apply_hierf_trans)

In [0]:
hierf_df.insert(0, "countyid", None)
hierf_df.head()

In [0]:
data_loc = hdf.get("data_loc")
loc_hierf = data_loc.join(hierf_df, how="inner")
loc_hierf.head()

In [0]:
hdf.put("loc_hierf", loc_hierf)
hdf.put("bayenv_df", loc_hierf)

In [0]:
loc_hierf['county_state'] = loc_hierf.apply(lambda row: "%s_%s" % (row.county, row.state), axis=1)
usable_counties = set()
county_counts = loc_hierf.county_state.value_counts()
county_counts = county_counts.sort(inplace=False, ascending=False)
for c in county_counts.index:
    print c, county_counts[c]
for c in county_counts.index:
    if county_counts[c] >=5:
        usable_counties.add(c)
usable_counties = sorted(list(usable_counties))

county_id = {}
for i, county in enumerate(usable_counties):
    county_id[county] = i+1
county_id

loc_hierf['usable'] = loc_hierf.apply(lambda row: row.county_state in county_id, axis=1)

drop = loc_hierf[loc_hierf.usable==False]

loc_hierf = loc_hierf.drop(drop.index)

loc_hierf['countyid'] = loc_hierf.apply(lambda row: county_id[row.county_state], axis=1)

loc_hierf.head()

In [0]:
dill.dump(county_id, open("county_id.dill","w"))

In [0]:
loc_hierf.ix[:,4:-2].to_csv("hierf.txt", sep="\t", header=True, index=False)

I've already run this for you, but just in case you wanted to do something like it in the future, the code is below.  Expect it to take a few hours, or days/weeks for huge datasets.


```
library(hierfstat)
data = read.table("hierf.txt", header=T, sep="\t")
data = data[order(data$countyid),]
levels = data.frame(data$countyid)
loci = data[,2:ncol(data)]
bs = basic.stats(data)
saveRDS(bs, "basic_stats.rds")
res = varcomp.glob(levels=levels, loci=loci, diploid=T)
saveRDS(res, "hierf.rds")
```

####Let's read that hierfstat into R, so we can get it into Python.  

No seriously, it's better.  Trust me.

In [0]:
%%R
bs = readRDS("basic_stats.rds")
res = readRDS("hierf.rds")

In [0]:
res = com.convert_robj(ro.r('res'))
bs = com.convert_robj(ro.r('bs'))
Fis = bs['Fis']
Hs = bs['Hs']
pop_freq_temp = bs['pop.freq']
pop_freq = {}
perloc = bs['perloc']
n_ind_samp = bs['n.ind.samp']
Ho = bs['Ho']
overall = bs['overall']

for df in [Fis, Hs, perloc, n_ind_samp, Ho]:
    df.index = [x[1:].replace(".","-") for x in df.index]

for locus, data in pop_freq_temp.items():
    if len(data) == 2:
        data.index = ['p','q']
    else:
        data.index = ['p']
    pop_freq[locus[1:].replace(".", "-")] = data

Ho = Ho.T
perloc = perloc.T
n_ind_samp = n_ind_samp.T
Hs = Hs.T
Fis = Fis.T

####You know have matrices for Ho, perloc, n_ind_samp, Hs, and Fis from hierfstat available as `DataFrame`s.  

Add some cells and explore them with head()

In [0]:
loc_df = res['loc']
F_df = res['F']
overall_df = res['overall']

####What data is contained in the F matrix (`F_df`)?  

In [0]:
F_df

####We can compute $F_{ST}$ for each site, as well, from the variance components in `loc_df`.

In [0]:
def compute_fst(series):
    Va = series[0]
    Vt = sum(series)
    return Va/Vt

In [0]:
loci_fst = loc_df.apply(compute_fst, axis=1).dropna()
loci_fst.index = [x[1:].replace(".", "-") for x in loci_fst.index]

In [0]:
plt.hist(loci_fst, bins=50)
plt.xlim(-.03, .5)
plt.title("Fst for %d loci $(\mu=%.3f \pm %.4f) [%.3f, %.3f]$" % (len(loci_fst),
                                                                  np.mean(loci_fst),
                                                                  np.std(loci_fst),
                                                                  np.min(loci_fst),
                                                                  np.max(loci_fst)))
plt.show()

In [0]:
loci_fst[loci_fst>0.1]

####Let's now join up our phenotype data with our hierfstat data.

You'll often find that you'll have to do this, especially when working in collaboration across groups.  It's not ideal, as you'll see, because not everything matches up.  Oh well, moving on.

In [0]:
def get_phenotype(row):
    return np.max(pheno[(pheno.Longitude==row.long) & (pheno.Latitude==row.lat)])

pheno = hdf.get("pheno")
trait = loc_hierf.apply(get_phenotype, axis=1)

In [0]:
trait.head()

####Inner, outer (left, right) joins are possible in Python, too.

In [0]:
trait_loc_hierf = trait.join(loc_hierf, how="inner")

####Let's drop populations that do not have data for your favorite trait.

Remember saving that file?

In [0]:
trait_name = dill.load(open("trait_name.dill"))
trait_complete = trait_loc_hierf.drop(trait_loc_hierf[np.isnan(trait_loc_hierf[trait_name])].index)

####Let's set up our PCA matrix to only have the axes that relate to our population structure, and save the rest of the data out for later.

In [0]:
pca_cov = x_drop.ix[:,0:14]

In [0]:
hdf.put("trait_complete", trait_complete)
hdf.put("x_drop", x_drop)
hdf.put("pca_cov", pca_cov)

In [0]:
trait_complete

In [0]:
for key, df in {"Fis":Fis, 
                "Hs":Hs, 
                "perloc": perloc, 
                "n_ind_samp":n_ind_samp, 
                "Ho":Ho}.items():
    hdf.put(key, df)

In [0]:
hdf.put("trait_complete", trait_complete)